In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import seaborn as sns
sns.set_style('whitegrid')

import warnings
warnings.simplefilter('ignore')


In [ ]:
df = pd.read_csv('/ihme/homes/rmbarber/covid-19/elastispliner/2020_06_07.08/model_data.csv')
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values(['location_id', 'date'])

first_date = df['date'] == df.groupby('location_id')['date'].transform(min)
df['Daily death rate (HFR)'] = np.diff(df['Predicted death rate (HFR)'], prepend=np.nan)
df.loc[first_date, 'Daily death rate (HFR)'] = df['Predicted death rate (HFR)']
df['Daily hospitalization rate'] = np.diff(df['Hospitalization rate'], prepend=np.nan)
df.loc[first_date, 'Daily hospitalization rate'] = df['Hospitalization rate']

df['HFR'] = df['Daily hospitalization rate'] / df['Daily death rate (HFR)']

df = df.loc[~df['HFR'].isnull()]

In [ ]:
with PdfPages('/ihme/homes/rmbarber/covid-19/cases_hospitalizations.pdf') as pdf:
    for location_id in df['location_id'].unique():
        plot_df = df.loc[df['location_id'] == location_id].reset_index(drop=True)

        plt.figure(figsize=(11, 8.5))
        plt.plot(plot_df['date'], plot_df['HFR'])
        plt.yticks(fontsize=10)
        plt.xticks(rotation=60, fontsize=10)
        plt.title(plot_df['location_name'][0], fontsize=20)
        plt.ylabel('HFR', fontsize=14)
        plt.xlabel('Date', fontsize=14)
        plt.tight_layout()
        pdf.savefig()
